In [ ]:
""" This part of code is to demonstraste the use of roberta to perform the fill
     in the blanks task """


# Import the pipeline module from the transformers library
from transformers import pipeline

# Load the model and tokenizer for masked language modeling using the pipeline
fill_mask = pipeline("fill-mask", model="roberta-base")

# Define the text with a blank to fill\
# <mask> used to define the blanks in the sentence
text = "The capital of France is <mask>."

# Generate multiple predictions for the masked word
# top_k=1 means Top predictions will be considered
# we can change the value of top_k to get other results also
predictions = fill_mask(text, top_k=2)

# Print the results (assuming 'score' is present in the output)
print("Predicted words and scores:")
# Iterate through each prediction and print the predicted word and its score
for prediction in predictions:
    print(f"- {prediction['token_str']}: {prediction['score']:.2f}")


In [ ]:
""" This part code is the automation of the above code.
    It takes the input fron the excel sheet of fill in the blanks questions
    It produces the result in the form of excel sheet contaion the the answers to questions"""

# Import necessary modules
import transformers
from transformers import pipeline
import pandas as pd
import os

# Load the fill-mask pipeline for RoBERTa model
fill_mask = pipeline("fill-mask", model="roberta-base")

# Load questions from an Excel file
try:
    df = pd.read_excel("/content/5000 Q.xlsx")  # Replace with the path to your Excel file
    questions = df["Fill in the blanks"]  #Enter the column that contains the questions
except Exception as e:
    print("Error occurred while loading the Excel file:", e)
    exit()

# Process each question and fill in the blank
filled_words = []
for question in questions:
    try:
        # Get the first prediction for each question
        prediction = fill_mask(question, top_k=1)[0]["token_str"]
        filled_words.append(prediction.strip())
    except Exception as e:
        print("Error occurred while processing the question:", e)
        filled_words.append("Error")

# Add filled words to the DataFrame (optional)
df["Filled Word"] = filled_words  # Add a new column with the filled words

# Specify the path for the output Excel file
output_file_path = "/content/filled_words.xlsx"

# Save filled words (or DataFrame with filled word column) to a new Excel file
try:
    # Choose between saving filled words as a list or within the DataFrame
    if not 'Filled Word' in df.columns:  # If the column doesn't exist, save filled words as a separate list
        pd.Series(filled_words).to_excel(output_file_path, index=False)
    else:  # If the column exists, save DataFrame with "Filled Word" column
        df.to_excel(output_file_path, index=False)
    print("Filled words saved to:", output_file_path)
except Exception as e:
    print("Error occurred while saving the Excel file:", e)
    # If an error occurs while saving, create a backup file with filled words
    output_file_path = "/content/filled_words_backup.xlsx"
    if not 'Filled Word' in df.columns:  # Save filled words as a separate list to the backup file
        pd.Series(filled_words).to_excel(output_file_path, index=False)
    else:  # Save DataFrame with "Filled Word" column to the backup file
        df.to_excel(output_file_path, index=False)
    print("Filled words saved to backup file:", output_file_path)
